In [11]:
!pip install keras-tuner


In [12]:
import pandas as pd
from pandas import DataFrame
import numpy as np

df_train = pd.read_csv('/content/deceptive_training.csv')
df_test = pd.read_csv('/content/deceptive_testing.csv')

In [13]:
### Remplazar los truthful con 0 y deceptive con 1 en la columna deceptive
df_train = df_train.replace({'deceptive': {'truthful': 0, 'deceptive': 1}})
df_test = df_test.replace({'deceptive': {'truthful': 0, 'deceptive': 1}})

In [14]:
train_dec_a = df_train.iloc[:, 0]
#validation_dec_a = validation_set.iloc[:, 0]
test_dec_a = df_test.iloc[:, 0]

train_dec_a.columns = ['train_dec_a']
#validation_dec_a.columns = ['validation_dec_a']
test_dec_a.columns = ['test_dec_a']

In [15]:
x_train = ()
y_train = ()

for index, row in df_train.iterrows():
    res = list(map(float, row['text'].split(' ')))
    res = np.asarray(res, dtype=np.float32)
    x_train = x_train + (res,)
    y_train = y_train + (row['deceptive'],)

"""
x_validation = ()
y_validation = ()

for index, row in validation_set.iterrows():
    res = list(map(float, row['text'].split(' ')))
    res = np.asarray(res, dtype=np.float32)
    x_validation = x_validation + (res,)
    y_validation = y_validation + (row['deceptive'],)
"""
x_test = ()
y_test = ()

for index, row in df_test.iterrows():
    res = list(map(float, row['text'].split(' ')))
    res = np.asarray(res, dtype=np.float32)
    x_test = x_test + (res,)
    y_test = y_test + (row['deceptive'],)

In [16]:
import numpy as np
x_train, y_train = np.asarray(x_train), np.asarray(y_train)
#x_validation, y_validation = np.asarray(x_validation), np.asarray(y_validation)
x_test, y_test = np.asarray(x_test), np.asarray(y_test)

x_train = x_train.astype('float32')
#x_validation = x_validation.astype('float32')
x_test = x_test.astype('float32')

In [17]:
# reshape inputs for LSTM [samples, timesteps, features]
x_train = x_train.reshape(x_train.shape[0], 1, x_train.shape[1]) # X_train 
print("Training data shape:", x_train.shape)
#x_validation = x_validation.reshape(x_validation.shape[0], 1, x_validation.shape[1]) # X_train 
#print("Validation data shape:", x_validation.shape)
x_test = x_test.reshape(x_test.shape[0], 1, x_test.shape[1])     # X_test 
print("Test data shape:", x_test.shape)

Training data shape: (597, 1, 300)
Test data shape: (198, 1, 300)


## Add GPU to be used

In [18]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

## Keras Tuner model

In [19]:
## Aparte del modelo vamos a definir los hiperparametros construyendo el hypermodelo
## Vamos a utilizar o bien una funccion builder o utilizar una subclase HyperModel usando la Keras Tuner 

import keras
from keras.layers import Dense, Input, Conv2D, Flatten, Reshape, Conv2DTranspose, LSTM, TimeDistributed, RepeatVector, Dropout
from tensorflow.keras import models, layers
from keras.models import Model, Sequential
from keras.utils import plot_model
from keras import backend as BE
from keras import regularizers
import kerastuner as kt
from kerastuner import HyperModel
from kerastuner import Hyperband
from kerastuner import BayesianOptimization
from kerastuner.tuners import RandomSearch

import numpy as np
import matplotlib.pyplot as plt


class RegressionHyperModel(HyperModel):
    def __init__(self, input_shape):
        self.input_shape = input_shape
    def build(self, hp):
        model = Sequential()
        model.add(
            layers.LSTM(300,
                #units=hp.Int('units', 8, 64, 4, default=8),
                activation=hp.Choice(
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'),
                return_sequences=True,
                input_shape=input_shape
            )
        )
        """
        for i in range(hp.Int('num_layers', 1, 10)):
            model.add(layers.Dense(units=hp.Int('lay_' + str(i),
                                                min_value=16,
                                                max_value=256,
                                                step=16),
                                    activation=hp.Choice(
                                        'dense_activation',
                                        values=['relu', 'tanh', 'sigmoid'],
                                        default='relu')))
        """
        L1=model.add(
            layers.LSTM(
                units=hp.Int('units_2', min_value=16, max_value=256, step=16), # 'units_2', 16, 32, 64, default=32),
                activation=hp.Choice(
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'),
                    kernel_regularizer=regularizers.l2(hp.Choice('regularizers.l2', values=[0.00, 0.01, 0.05, 0.1])), # Valorarlo... 
                return_sequences=True,
                input_shape=input_shape
            )
        ) 
        
        L2=model.add(
            layers.LSTM(
                units=hp.Int('units_3', min_value=2, max_value=32, step=2),              #'units_3', 2, 4, 8, default=4
                activation=hp.Choice( 
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'),
                return_sequences=True,
                input_shape=input_shape
            )
        )  
        
        
        L3=model.add(
            layers.LSTM(
                units=hp.Int('units_4', min_value=2, max_value=32, step=2),              #'units_3', 2, 4, 8, default=4
                activation=hp.Choice( 
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'),
                return_sequences=True,
                input_shape=input_shape
            )
        ) 
        
        L4=model.add(
            layers.LSTM(
                units=hp.Int('units_5', min_value=16, max_value=256, step=16),        #'units_4', 16, 32, 64, default=32
                activation=hp.Choice(
                    'dense_activation',
                    values=['relu', 'tanh', 'sigmoid'],
                    default='relu'),
                return_sequences=True,
                input_shape=input_shape
            )
        )
        model.add(
            layers.Dropout(
                hp.Float(
                    'dropout',
                    min_value=0.0,
                    max_value=0.1,
                    default=0.005,
                    step=0.01)
            )
        )
        
        model.add(layers.Dense(300))
        optimizer = hp.Choice('optimizer', ['adam', 'sgd', 'rmsprop','nadam', 'adadelta',
                                            'adagrad', 'adamax','ftrl'])
        model.compile(
            optimizer=optimizer,loss='mse',metrics=['mse']
        )
        
        return model

In [20]:
input_shape = (x_train.shape[1],x_train.shape[2],)
hypermodel = RegressionHyperModel(input_shape)

## RandomSearch

In [21]:
tuner_rs = RandomSearch(
            hypermodel,
            objective='mse',
            seed=10,
            max_trials= 2,
            project_name='opiniondetection',
            overwrite=True,
            executions_per_trial=2)

In [22]:
tuner_rs.search(x_train, x_train, epochs=20, validation_split = 0.2, verbose=0)

INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner_rs.search_space_summary()

In [ ]:
tuner_rs.results_summary()

In [ ]:
model = tuner_rs.get_best_models(num_models=1)[0]
loss, mse = model.evaluate(x_train, x_train)

In [ ]:
tuner_rs.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values

In [ ]:
model = tuner_rs.get_best_models(num_models=1)[0]
mse_rs = model.evaluate(x_test, x_test)[1]
print('Random search MSE: ', mse_rs)